In [2]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder

In [180]:
train_path = "H:\\Hobby\\ML Dataset\\titanic\\train.csv"
test_path = "H:\\Hobby\\ML Dataset\\titanic\\test.csv"

In [181]:
train = pd.read_csv(train_path)
test = pd.read_csv(test_path)

y=train['Survived']
train=train.drop(['Survived','PassengerId', 'Ticket'], axis=1)
test = test.drop(['Ticket'], axis=1)

In [182]:
train['Age'] = train['Age'].fillna(80)
test['Age'] = test['Age'].fillna(80)
train['Embarked'] = train['Embarked'].fillna('S')
test['Fare'] = test['Fare'].fillna(32)

In [183]:
train['Cabin'] = train['Cabin'].str[:1]
cabin_mapping = {"A": 0, "B": 0.4, "C": 0.8, "D": 1.2, "E": 1.6, "F": 2, "G": 2.4, "T": 2.8}
train['Cabin'] = train['Cabin'].map(cabin_mapping)
test['Cabin'] = test['Cabin'].map(cabin_mapping)

In [184]:
train_test_data = [train, test] 

for dataset in train_test_data:
    dataset['Title'] = dataset['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
    
title_mapping = {"Mr": 0, "Miss": 1, "Mrs": 2, 
                 "Master": 3, "Dr": 3, "Rev": 3, "Col": 3, "Major": 3, "Mlle": 3,"Countess": 3,
                 "Ms": 3, "Lady": 3, "Jonkheer": 3, "Don": 3, "Dona" : 3, "Mme": 3,"Capt": 3,"Sir": 3 }
for dataset in train_test_data:
    dataset['Title'] = dataset['Title'].map(title_mapping)

In [185]:
train = train.drop(['Name'], axis=1)
test = test.drop(['Name'], axis=1)

In [186]:
cols = (train.dtypes=='object')
cols = list(cols[cols].index)

encoder = LabelEncoder()

for col in cols:
    train[col] = encoder.fit_transform(train[col])
    test[col] = encoder.transform(test[col])

In [187]:
train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Title
0,3,1,22.0,1,0,7.2500,NaN,2,0
1,1,0,38.0,1,0,71.2833,0.8,0,2
2,3,0,26.0,0,0,7.9250,NaN,2,1
3,1,0,35.0,1,0,53.1000,0.8,2,2
4,3,1,35.0,0,0,8.0500,NaN,2,0


In [188]:
imputer = SimpleImputer()

temp = test.copy()
test = test.drop(['PassengerId'], axis=1)
temp_test = test.copy()

train = pd.DataFrame(imputer.fit_transform(train))
test = pd.DataFrame(imputer.transform(test))

train.columns = temp_test.columns
test.columns = temp_test.columns

In [208]:
xTrain, xVal, yTrain, yVal = train_test_split(train, y)

greatest = 0;optimal=0
for x in range(1,31,2):
    knn = KNeighborsClassifier(n_neighbors=x)
    knn.fit(xTrain, yTrain)
    preds = knn.predict(xVal)
    score = accuracy_score(preds, yVal)
    if score>greatest:
        greatest=score
        optimal=x
    print(score, x)
print('Smallest: %.3f, Optimal: %d' %(greatest,optimal))

0.7130044843049327 1
0.726457399103139 3
0.7309417040358744 5
0.7174887892376681 7
0.7085201793721974 9
0.7040358744394619 11
0.7085201793721974 13
0.6816143497757847 15
0.6860986547085202 17
0.672645739910314 19
0.6367713004484304 21
0.6367713004484304 23
0.6412556053811659 25
0.6367713004484304 27
0.6367713004484304 29
Smallest: 0.731, Optimal: 5


In [210]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(train, y)
preds = knn.predict(test)

output = pd.DataFrame({'PassengerId': temp.PassengerId,
                      'Survived': preds})

output.to_csv('2ndSubmission.csv', index=False)